In [12]:
import warnings 
warnings.filterwarnings('ignore')

In [13]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier

In [14]:
df=pd.read_csv("results.csv")

In [15]:
df_match=df.copy() 

In [16]:
df_match.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [17]:
df_match = df_match[df_match['date']>= '1986-05-31']

In [18]:
df_match.reset_index(drop = True, inplace = True)

In [21]:
tournament = pd.DataFrame(df_match['tournament'].value_counts())

In [22]:
tournament.reset_index(inplace = True)

In [23]:
years = []
for date in df_match.date:
    years.append(int(str(date)[0:4]))

In [24]:
New_Dataset_part_1=pd.DataFrame(list(zip(years,df_match.values[:,7],df_match.values[:,1],df_match.values[:,2],df_match.values[:,3],df_match.values[:,4])),columns=["year","Country","team_1","team_2","team_1_score","team_2_score"])
New_Dataset_part_2=pd.DataFrame(list(zip(years,df_match.values[:,7],df_match.values[:,2],df_match.values[:,1],df_match.values[:,4],df_match.values[:,3])),columns=["year","Country","team_1","team_2","team_1_score","team_2_score"])
New_Dataset=pd.concat([New_Dataset_part_1,New_Dataset_part_2],axis=0)
New_Dataset =New_Dataset.sample(frac=1).reset_index(drop=True)

In [25]:
teams_1=New_Dataset.team_1.unique()
contries=New_Dataset.Country.unique()
all_countries=np.unique(np.concatenate((teams_1,contries), axis=0))

In [26]:
Y= New_Dataset.iloc[:,4:6] #Training targets (team_1_score and team_2_score)
categorized_data=New_Dataset.iloc[:,0:4].copy() #Traing features

label_encoder = preprocessing.LabelEncoder()

label_encoder.fit(all_countries)
categorized_data['team_1']=label_encoder.transform(categorized_data['team_1'])
categorized_data['team_2']=label_encoder.transform(categorized_data['team_2'])
categorized_data['Country']=label_encoder.transform(categorized_data['Country'])

categorized_data['team_1']=categorized_data['team_1'].astype("category")
categorized_data['team_2']=categorized_data['team_2'].astype("category")
categorized_data['Country']=categorized_data['team_2'].astype("category")

X=categorized_data

In [55]:
model = MultiOutputRegressor(RandomForestClassifier(n_estimators = 15))
model.fit(X,Y)

MultiOutputRegressor(estimator=RandomForestClassifier(n_estimators=15))

In [57]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [58]:
def select_winning_team(probability_array):
    prob_lst=[round(probability_array[0][i],3) for i in range(2)]
    if (prob_lst[0]>prob_lst[1]):
        out=0
    elif (prob_lst[0]<prob_lst[1]):
        out=1
    elif (prob_lst[0]==prob_lst[1]):
        out=2
    return out,prob_lst

In [59]:
# select team group match
def select_team_groupmatch(selectteams):
    select_team_wins_dct={}
    select_goal_scored_dct={}
    select_goal_against_dct={}
    select_win_dct={}
    select_draw_dct={}
    select_lost_dct={}
    for i in range(len(selectteams)):
        j=i+1
        select_team_1 = selectteams[i]
        select_team_wins=0
        while j < len((selectteams)):
            select_team_2=selectteams[j]
            select_team_lst=[select_team_1,select_team_2]
            select_Input_vector=np.array([[year,host_num, clabel[select_team_1] , clabel[select_team_2]]])
            res=model.predict(select_Input_vector)

            select_win,prob_lst = select_winning_team(res)
            select_goal_scored_dct[select_team_1] = select_goal_scored_dct.get(select_team_1,0)+prob_lst[0]
            select_goal_scored_dct[select_team_2] = select_goal_scored_dct.get(select_team_2,0)+prob_lst[1]

            select_goal_against_dct[select_team_1] = select_goal_against_dct.get(select_team_1,0)+prob_lst[1]
            select_goal_against_dct[select_team_2] = select_goal_against_dct.get(select_team_2,0)+prob_lst[0]

            try:
                if (select_win)==0:
                    select_team_wins_dct[select_team_1] = select_team_wins_dct.get(select_team_1,0)+3
                    select_team_wins_dct[select_team_2] = select_team_wins_dct.get(select_team_2,0)
                    
                    select_win_dct[select_team_1] = select_win_dct.get(select_team_1,0)+1
                    select_win_dct[select_team_2] = select_win_dct.get(select_team_2,0)
                    select_lost_dct[select_team_2] = select_lost_dct.get(select_team_2,0)+1
                    select_lost_dct[select_team_1] = select_lost_dct.get(select_team_1,0)
                    select_draw_dct[select_team_2] = select_draw_dct.get(select_team_2,0)
                    select_draw_dct[select_team_1] = select_draw_dct.get(select_team_1,0)

                elif (select_win)==1:
                    select_team_wins_dct[select_team_2] = select_team_wins_dct.get(select_team_2,0)+3
                    select_team_wins_dct[select_team_1] = select_team_wins_dct.get(select_team_1,0)
                    
                    select_win_dct[select_team_2] = select_win_dct.get(select_team_2,0)+1
                    select_win_dct[select_team_1] = select_win_dct.get(select_team_1,0)
                    select_lost_dct[select_team_1] = select_lost_dct.get(select_team_1,0)+1
                    select_lost_dct[select_team_2] = select_lost_dct.get(select_team_2,0)
                    select_draw_dct[select_team_1] = select_draw_dct.get(select_team_1,0)
                    select_draw_dct[select_team_2] = select_draw_dct.get(select_team_2,0)

            except IndexError:
                select_team_wins_dct[select_team_1] = select_team_wins_dct.get(select_team_1,0)+1
                select_team_wins_dct[select_team_2] = select_team_wins_dct.get(select_team_2,0)+1
                
                select_draw_dct[select_team_1] = select_draw_dct.get(select_team_1,0)+1
                select_draw_dct[select_team_2] = select_draw_dct.get(select_team_2,0)+1
                
                select_win_dct[select_team_1] = select_win_dct.get(select_team_1,0)
                select_lost_dct[select_team_1] = select_lost_dct.get(select_team_1,0)
              
                select_win_dct[select_team_2] = select_win_dct.get(select_team_2,0)
                select_lost_dct[select_team_2] = select_lost_dct.get(select_team_2,0)
                    
            j=j+1
    select_results=[select_win_dct,select_draw_dct,select_lost_dct,select_team_wins_dct,select_goal_scored_dct,select_goal_against_dct]

    return select_results

In [60]:
##Group stage Matches
def all_group_stage():
    Group_standings={}
    for grp_name in list(Groups.keys()):
        probable_countries=Groups[grp_name]
        team_wins_dct={}
        goal_scored_dct={}
        goal_against_dct={}
        win_dct={}
        draw_dct={}
        lost_dct={}
        for i in range(len(probable_countries)):
            j=i+1
            team_1=probable_countries[i]
            team_wins=0
            while j<len((probable_countries)):
                team_2=probable_countries[j]
                team_lst=[team_1,team_2]
                Input_vector=np.array([[year,stadium_num,clabel[team_1],clabel[team_2]]])
                res=model.predict(Input_vector)

                win,prob_lst=select_winning_team(res)
                goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
                goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]

                goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
                goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]

                try:
                    if (win)==0:
                        team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+3
                        team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    
                        win_dct[team_1] = win_dct.get(team_1,0)+1
                        win_dct[team_2] = win_dct.get(team_2,0)
                        lost_dct[team_2] = lost_dct.get(team_2,0)+1
                        lost_dct[team_1] = lost_dct.get(team_1,0)
                        draw_dct[team_2] = draw_dct.get(team_2,0)
                        draw_dct[team_1] = draw_dct.get(team_1,0)

                    elif (win)==1:
                        team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+3
                        team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    
                        win_dct[team_2] = win_dct.get(team_2,0)+1
                        win_dct[team_1] = win_dct.get(team_1,0)
                        lost_dct[team_1] = lost_dct.get(team_1,0)+1
                        lost_dct[team_2] = lost_dct.get(team_2,0)
                        draw_dct[team_1] = draw_dct.get(team_1,0)
                        draw_dct[team_2] = draw_dct.get(team_2,0)

                except IndexError:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
                
                    draw_dct[team_1] = draw_dct.get(team_1,0)+1
                    draw_dct[team_2] = draw_dct.get(team_2,0)+1
                
                    win_dct[team_1] = win_dct.get(team_1,0)
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                
                    win_dct[team_2] = win_dct.get(team_2,0)
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    
                j=j+1
        group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
        Group_standings[grp_name]=group_results

    return group_results, Group_standings

In [61]:
def round_of_16():
        ##Round of 16 Section_1
        qualified_teams_1=[]
        standings=list(Group_standings.keys())
        i=0
        while i < (len(standings)):
                A_team= sorted(Group_standings[standings[i]][3].items(), key=lambda x: x[1], reverse=True)
                team_1=A_team[0][0]
                B_team= sorted(Group_standings[standings[i+1]][3].items(), key=lambda x: x[1], reverse=True)
                team_2=B_team[1][0]
        
                team_lst=[team_1,team_2]
    
                Input_vector=np.array([[year,host_num,clabel[team_1],clabel[team_2]]])
                res=model.predict(Input_vector)
                win,_=select_winning_team(res)

                try:
                        qualified_teams_1.append(team_lst[win])
                except IndexError:
                        winning_team=random.choice(team_lst)
                        qualified_teams_1.append(winning_team)
                i=i+2

        ##Round of 16 Section_2
        qualified_teams_2=[]
        standings=list(Group_standings.keys())
        i=0
        while i < (len(standings)):
                A_team= sorted(Group_standings[standings[i]][3].items(), key=lambda x: x[1], reverse=True)
                team_1=A_team[1][0]
                B_team= sorted(Group_standings[standings[i+1]][3].items(), key=lambda x: x[1], reverse=True)
                team_2=B_team[0][0]
    
                team_lst=[team_1,team_2]
    
                Input_vector=np.array([[year,host_num,clabel[team_1],clabel[team_2]]])
                res=model.predict(Input_vector)
                win,_=select_winning_team(res)

                try:
                        qualified_teams_2.append(team_lst[win])
            
                except IndexError:
                        winning_team=random.choice(team_lst)
                        qualified_teams_2.append(winning_team)
                i=i+2
                
        return qualified_teams_1, qualified_teams_2


In [62]:
#Quarter Finals

def quarter():
    Semifinal_teams=[]
    i=0
    while i < (len(qualified_teams_1))-1:
        team_1= qualified_teams_1[i]
        team_2= qualified_teams_1[i+1]
    
        team_lst=[team_1,team_2]

        Input_vector=np.array([[year,host_num,clabel[team_1],clabel[team_2]]])
        res=model.predict(Input_vector)
        win,_=select_winning_team(res)

        try:
            Semifinal_teams.append(team_lst[win])
            
        except IndexError:
            winning_team=random.choice(team_lst)
            Semifinal_teams.append(winning_team)
        i=i+2
    
    i=0
    while i < (len(qualified_teams_2))-1:
        team_1= qualified_teams_2[i]
        team_2= qualified_teams_2[i+1]

        team_lst=[team_1,team_2]
    
        Input_vector=np.array([[year,host_num,clabel[team_1],clabel[team_2]]])
        res=model.predict(Input_vector)
        win,_=select_winning_team(res)

        try:
            Semifinal_teams.append(team_lst[win])
            
        except IndexError:
            winning_team=random.choice(team_lst)
            Semifinal_teams.append(winning_team)
        i=i+2

    return Semifinal_teams

In [63]:
#Semi Finals

def semi_final():
    final_teams=[]
    third_place_match_teams=[]
    i=0
    while i < (len(Semifinal_teams))-1:
        team_1= Semifinal_teams[i]
        team_2= Semifinal_teams[i+1]
    
        team_lst=[team_1,team_2]
    
        Input_vector=np.array([[year,host_num,clabel[team_1],clabel[team_2]]])
        res=model.predict(Input_vector)
        win,_=select_winning_team(res)

        try:
            final_teams.append(team_lst[win])
            third_place_match_teams.append(team_lst[(win+1)%2])

            
        except IndexError:
            winning_team=random.choice(team_lst)
            final_teams.append(winning_team)
            team_lst.remove(winning_team)
            third_place_match_teams.append(team_lst[0])
        i=i+2
    
    return final_teams, third_place_match_teams

In [64]:
#Third Place match

def third_place():
    team_1= third_place_match_teams[1]
    team_2= third_place_match_teams[0]
    
    team_lst=[team_1,team_2]
    
    Input_vector=np.array([[year,host_num,clabel[team_1],clabel[team_2]]])
    res=model.predict(Input_vector)
    win,_=select_winning_team(res)

    try:
        place_3=team_lst[win]
            
    except IndexError:
        winning_team=random.choice(team_lst)
        place_3 = winning_team
    
    return place_3


In [65]:
# finals

def final():
    team_1= final_teams[1]
    team_2= final_teams[0]
    
    team_lst=[team_1,team_2]
    
    Input_vector=np.array([[year,host_num,clabel[team_1],clabel[team_2]]])
    res=model.predict(Input_vector)
    win,_=select_winning_team(res)

    try:
        winner=team_lst[win]
        place_2=team_lst[(win+1)%2]
            
    except IndexError:
        winning_team=random.choice(team_lst)
        winner=winning_team
    
        team_lst.remove(winning_team)
        place_2=team_lst[0]
    
    return winner, place_2 


In [66]:
Group_A= ["Qatar","Ecuador","Senegal","Netherlands"]
Group_B= ["England","Iran","United States","Wales"]
Group_C= ["Argentina","Saudi Arabia","Mexico","Poland"]
Group_D= ["France","Australia","Denmark","Tunisia"]
Group_E= ["Spain","Costa Rica","Germany","Japan"]
Group_F= ["Belgium","Canada","Morocco","Croatia"]
Group_G= ["Brazil","Serbia","Switzerland","Cameroon"]
Group_H= ["Portugal","Ghana","Uruguay","South Korea"]
Groups={"Group A":Group_A,"Group B":Group_B,"Group C":Group_C,"Group D":Group_D,"Group E":Group_E,"Group F":Group_F,"Group G":Group_G,"Group H":Group_H}

In [67]:
countries = ["Qatar","Ecuador","Senegal","Netherlands","England","Iran","United States","Wales","Argentina","Saudi Arabia","Mexico","Poland",
            "France","Australia","Denmark","Tunisia", "Spain","Costa Rica","Germany","Japan", "Belgium","Canada","Morocco","Croatia",
            "Brazil","Serbia","Switzerland","Cameroon","Portugal","Ghana","Uruguay","South Korea"]

In [68]:
countries_label = label_encoder.transform(countries)
clabel = pd.DataFrame(zip(countries, countries_label), columns = ['country', 'label'])
clabel['label'] = clabel['label'].astype('category')

In [69]:
clabel = {name:value for name, value in zip(countries, list(clabel['label']))}

In [70]:
year = 2022
stadium = "Qatar"
stadium_num = clabel[stadium]
host_num=stadium_num

In [71]:
selectteam = input()

if selectteam in Group_A:
    selectteams = Group_A
elif selectteam in Group_B:
    selectteams = Group_B
elif selectteam in Group_C:
    selectteams = Group_C
elif selectteam in Group_D:
    selectteams = Group_D
elif selectteam in Group_E:
    selectteams = Group_E
elif selectteam in Group_F:
    selectteams = Group_F
elif selectteam in Group_G:
    selectteams = Group_G
elif selectteam in Group_H:
    selectteams = Group_H

In [72]:
group_results, Group_standings = all_group_stage()

In [73]:
select_group_result = select_team_groupmatch(selectteams)
group_to_16 = sorted(select_group_result[3].items(), key = lambda x: x[1], reverse=True)[0:2]
group_to_16 = list(zip(*group_to_16))[0]

if selectteam not in group_to_16:
    endselect = '팀은 조별 경기에서 떨어졌습니다.'
else:
    qualified_teams_1, qualified_teams_2 = round_of_16()
    if selectteam not in qualified_teams_1 + qualified_teams_2:
        endselect = "팀의 최종성적은 16강입니다"
    else:
        Semifinal_teams = quarter()
        if selectteam not in Semifinal_teams:
            endselect = "팀의 최종성적은 8강입니다"
        else:
            final_teams, third_place_match_teams = semi_final()
            if selectteam not in final_teams:
                endselect = "팀은 4위입니다"
                place_3 = third_place()
                if place_3 == selectteam:
                    endselect = "팀은 3위입니다"
            else:
                winner, place_2 = final()
                if winner == selectteam:
                    endselect = "팀은 우승입니다"
                else:
                    endselect = "팀은 2위입니다"

In [74]:
print(f" 예측할 팀 : {selectteam}\n 예측한 팀의 조편성 : {selectteams} \n 예측 결과 : {selectteam}{endselect}")

 예측할 팀 : South Korea
 예측한 팀의 조편성 : ['Portugal', 'Ghana', 'Uruguay', 'South Korea'] 
 예측 결과 : South Korea팀은 조별 경기에서 떨어졌습니다.
